# Data Cleaning

In [1]:
DATA_PATH = "../output/raw-csv"
OUTPUT_PATH = "../output/csv"
OUTPUT_STAT_PATH = "../output/csv-stat"

OUTLIER_THRESHOLDS = {
    "pH": (0, 14),
    "EC": (0, 2000),
    "Temp": (0, 50),
    "DO": (0, 20)
}

In [2]:

from pathlib import Path
from pandas import DataFrame, read_csv
from matplotlib import pyplot as plt
from logging import basicConfig, INFO
from dataclasses import dataclass
from json import dump as json_dump
from IPython.display import display

basicConfig(level=INFO)

from mp import mp_print, mp_exec

INFO:mp:Set start method to fork


In [3]:
DATA_PATH = Path(DATA_PATH).resolve()
OUTPUT_PATH = Path(OUTPUT_PATH).resolve()
OUTPUT_STAT_PATH = Path(OUTPUT_STAT_PATH).resolve()

OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
OUTPUT_STAT_PATH.mkdir(parents=True, exist_ok=True)

In [4]:
def iter_files():
    for file in filter(lambda x: x.is_file() and x.is_file(), DATA_PATH.iterdir()):
        yield file

In [5]:
@dataclass
class Stat:
    name: str
    total: int
    missing: int
    outliers: int
    valid: int
    threshold_outliers: int
    iqr_outliers: int
    ph_iqr: float
    ph_lb: float
    ph_up: float
    ec_iqr: float
    ec_lb: float
    ec_up: float
    temp_iqr: float
    temp_lb: float
    temp_up: float
    do_iqr: float
    do_lb: float
    do_up: float

def task(file: Path):
    station = file.name.split(".")[0]
    mp_print(f"Processing : {station}")
    
    stat = Stat(station,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0)
    df = read_csv(file)
    stat.total = df.size

    df = df[["Datetime", "pH", "EC", "Temp", "DO"]]

    df.dropna(inplace=True)
    stat.missing = stat.total - df.size

    size = df.size
    for column, (low, high) in OUTLIER_THRESHOLDS.items():
        df = df[(df[column] >= low) & (df[column] <= high)]
        stat.outliers += size - df.size
        size = df.size
    
    stat.threshold_outliers = stat.outliers

    Q1 = df[["pH", "EC", "Temp", "DO"]].quantile(0.25)
    Q3 = df[["pH", "EC", "Temp", "DO"]].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df = df[
        (df["pH"] >= lower_bound["pH"]) & (df["pH"] <= upper_bound["pH"]) &
        (df["EC"] >= lower_bound["EC"]) & (df["EC"] <= upper_bound["EC"]) &
        (df["Temp"] >= lower_bound["Temp"]) & (df["Temp"] <= upper_bound["Temp"]) &
        (df["DO"] >= lower_bound["DO"]) & (df["DO"] <= upper_bound["DO"])
    ]
    
    for column in ["pH", "EC", "Temp", "DO"]:
        setattr(stat, f"{column.lower()}_iqr", IQR[column])
        setattr(stat, f"{column.lower()}_lb", lower_bound[column])
        setattr(stat, f"{column.lower()}_ub", upper_bound[column])

    stat.iqr_outliers = size - df.size
    stat.outliers += stat.iqr_outliers
    stat.valid = stat.total - stat.missing - stat.outliers

    assert stat.total == stat.missing + stat.outliers + stat.valid
    assert stat.outliers == stat.threshold_outliers + stat.iqr_outliers

    return (station, df, stat)

In [6]:
data: dict[str, DataFrame] = {x: (y, z) for x, y, z in mp_exec(task, iter_files(), unorder=True)}

stats: list[Stat] = []

for station, values in data.items():
    df, stat = values
    print(f"Station : {station}, data : {df.shape}")
    stats.append(stat)
    df.to_csv(OUTPUT_PATH / f"{station}.csv", index=False)

Processing : ป่าสัก3สถานี แก่งคอย 2558-2563
Processing : วัง เกาะคา 2558-2563
Processing : แม่น้ำท่าจีน กระทุ่มแบน 2558-2563
Processing : ป่าสัก3สถานี นครหลวง 2558-2563
Processing : แม่น้ำท่าจีน นครชัยศรี 2558-2563
Processing : ยม โพทะเล 2558-2563
Processing : ป่าสัก3สถานี เสาไห้ 2558-2563
Processing : แม่น้ำท่าจีน บางเลน 2558-2563
Processing : แม่น้ำท่าจีน สองพี่น้อง 2558-2563
Processing : แม่น้ำท่าจีน หันคา 2558-2563
Processing : ยม สุโขทัย 2558-2563
Processing : ยม สามง่าม 2558-2563
Processing : แม่น้ำท่าจีน สุพรรณบุรี 2558-2563
Processing : แม่น้ำท่าจีน สามชุก 2558-2563


/tmp/ipykernel_1311119/2245389297.py:28: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = read_csv(file)


Processing : ปิง เชียงใหม่ 2558-2563
Processing : ปิง กำแพงเพชร 2558-2563
Processing : น่าน อุตรดิตถ์ 2558-2563
Processing : น่าน น่าน 2558-2563
Processing : น่าน พิษณุโลก 2558-2563
Station : ยม สุโขทัย 2558-2563, data : (5848, 5)
Station : แม่น้ำท่าจีน สองพี่น้อง 2558-2563, data : (55732, 5)
Station : แม่น้ำท่าจีน กระทุ่มแบน 2558-2563, data : (28962, 5)
Station : แม่น้ำท่าจีน หันคา 2558-2563, data : (46028, 5)
Station : ป่าสัก3สถานี นครหลวง 2558-2563, data : (65432, 5)
Station : ยม โพทะเล 2558-2563, data : (30458, 5)
Station : วัง เกาะคา 2558-2563, data : (60395, 5)
Station : แม่น้ำท่าจีน สามชุก 2558-2563, data : (6859, 5)
Station : ป่าสัก3สถานี แก่งคอย 2558-2563, data : (46463, 5)
Station : ป่าสัก3สถานี เสาไห้ 2558-2563, data : (57086, 5)
Station : แม่น้ำท่าจีน นครชัยศรี 2558-2563, data : (39759, 5)
Station : ยม สามง่าม 2558-2563, data : (54218, 5)
Station : แม่น้ำท่าจีน บางเลน 2558-2563, data : (63850, 5)
Station : แม่น้ำท่าจีน สุพรรณบุรี 2558-2563, data : (66404, 5)
Station : ปิง เ

In [7]:
stats_df = DataFrame([s.__dict__ for s in stats])
stats_df.set_index("name", inplace=True)
stats_df.sort_values("valid", ascending=False, inplace=True)
stats_df.to_csv(OUTPUT_STAT_PATH / "stat.csv", index=False)

display(stats_df[["total", "missing", "outliers", "threshold_outliers", "iqr_outliers", "valid"]])

,total,missing,outliers,threshold_outliers,iqr_outliers,valid
name,,,,,,
น่าน พิษณุโลก 2558-2563,441325,40625,68505,0,68505,332195
แม่น้ำท่าจีน สุพรรณบุรี 2558-2563,701946,354341,15585,0,15585,332020
ป่าสัก3สถานี นครหลวง 2558-2563,766656,391016,48480,0,48480,327160
แม่น้ำท่าจีน บางเลน 2558-2563,888714,498684,70780,0,70780,319250
วัง เกาะคา 2558-2563,476845,126490,48380,16520,31860,301975
ป่าสัก3สถานี เสาไห้ 2558-2563,637371,294991,56950,9990,46960,285430
แม่น้ำท่าจีน สองพี่น้อง 2558-2563,449022,108072,62290,820,61470,278660
ยม สามง่าม 2558-2563,389295,19150,99055,6400,92655,271090
น่าน อุตรดิตถ์ 2558-2563,480825,145375,69400,5840,63560,266050


In [8]:
display(stats_df[[f"{x}_{y}" for y in ["iqr", "lb", "up"] for x in ["ph", "ec", "temp", "do"]]].round(3))

,ph_iqr,ec_iqr,temp_iqr,do_iqr,ph_lb,ec_lb,temp_lb,do_lb,ph_up,ec_up,temp_up,do_up
name,,,,,,,,,,,,
น่าน พิษณุโลก 2558-2563,1.30,35.900,2.6,1.8,3.95,101.550,22.60,2.40,0.0,0.0,0.0,0.0
แม่น้ำท่าจีน สุพรรณบุรี 2558-2563,2.70,127.400,2.6,2.7,0.95,63.700,25.60,-1.75,0.0,0.0,0.0,0.0
ป่าสัก3สถานี นครหลวง 2558-2563,0.80,107.300,2.6,2.0,5.30,121.950,25.50,-0.20,0.0,0.0,0.0,0.0
แม่น้ำท่าจีน บางเลน 2558-2563,0.70,192.900,6.9,2.3,5.15,-69.650,14.35,-2.15,0.0,0.0,0.0,0.0
วัง เกาะคา 2558-2563,1.60,338.300,3.6,4.1,4.20,-498.950,21.40,-4.75,0.0,0.0,0.0,0.0
ป่าสัก3สถานี เสาไห้ 2558-2563,1.20,128.700,2.4,2.6,6.00,149.950,25.60,-0.00,0.0,0.0,0.0,0.0
แม่น้ำท่าจีน สองพี่น้อง 2558-2563,1.08,127.900,2.1,1.1,4.18,165.650,26.45,-0.25,0.0,0.0,0.0,0.0
ยม สามง่าม 2558-2563,0.60,68.900,4.5,2.4,5.50,110.750,13.45,-0.50,0.0,0.0,0.0,0.0
น่าน อุตรดิตถ์ 2558-2563,2.10,89.900,3.5,1.3,2.05,43.750,20.75,3.55,0.0,0.0,0.0,0.0
